In [1]:
!python -m pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-t3xv1ybq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-t3xv1ybq
  Resolved https://github.com/huggingface/transformers to commit 52daf4ec768fb9ffe84a0c373834172a7c54aecc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9800639 sha256=9a0a6a90ba38f6b04b4a4816e802097cba8fd8d935cc721fbe305b7b087fe539
  Stored in directory: /tmp/pip-ephem-wheel-cache-el1lfui7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [3]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [4]:
from PIL import Image
import requests

In [5]:
image = Image.open("/content/Pathology Lab Sample report.png")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "What is wrong with the this report."
            }
        ]
    }
]

text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("cuda")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [6]:
print(output_text)

['The report appears to be a complete blood count (CBC) test result for a 21-year-old male patient named Yash M. Patel. Here are the key points to note:\n\n1. **Test Title**: The test is labeled as "Complete Blood Count (CBC)".\n2. **Patient Details**: The patient\'s name is Yash M. Patel, age is 21 years, sex is male, and patient ID (PID) is 555.\n3. **Sample Details**: The sample was collected at 125, Shivam Bungalow, S G Road, Ahmedabad, and registered on 02 Dec, 2021.\n4. **Test Result**: The hemoglobin (Hb) level is 12.5 g/dL, which is considered low.\n5. **Reference Range**: The reference range for hemoglobin is 13.0 - 17.0 g/dL.\n6. **Blood Indices**: The packed cell volume (PCV) is 37.5, which is low (40 - 50%).\n7. **Mean Corpuscular Volume (MCV)**: The MCV is 72.12, which is within the reference range.\n8. **Red Blood Cell (RBC) Count**: The RBC count is 5.2, which is within the reference range (4.5 - 5.5 mill/cumm).\n9. **White Blood Cell (WBC) Count**: The WBC count is 9000